In [9]:
from openai import OpenAI
import pandas as pd
import jsbeautifier
import json
import time
import datetime

In [10]:
client = OpenAI()
temperature=1.0
max_tokens=4000
frequency_penalty=0.0

In [11]:
def generate_gpt4(title, system_prompt, user_prompt):
    message=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt.format(titulo=title)}]
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages = message,
        temperature = temperature,
        max_tokens = max_tokens,
        frequency_penalty = frequency_penalty
    )
    return response.choices[0].message.content

def save_json(path, data):
    with open(path, 'w', encoding='utf8') as fp:
        fp.write(jsbeautifier.beautify(json.dumps(data, ensure_ascii=False)))

# función para calcular el tiempo restante en función de la última ejecución
def print_reaming_time(start_time, total_trainings, current_iterations):
    iteration_time = time.time() - start_time
                
    remaining_time_estimate = iteration_time * (total_trainings - current_iterations - 1)

    days = int(remaining_time_estimate // (24 * 3600))
    hours = int((remaining_time_estimate % (24 * 3600)) // 3600)
    minutes = int((remaining_time_estimate % 3600) // 60)
    seconds = int(remaining_time_estimate % 60)
    current_iterations += 1
    print(f"[{datetime.datetime.now()}] Iteration {current_iterations}/{total_trainings} - Estimated remaining time: {days} days, {hours} hours, {minutes} minutes, {seconds} seconds")


# Generación para Patricio

In [4]:
system_prompt = """
We are going to do an experiment in which we are going to compare your creative writing skills with those of a prestigious novelist, Patricio Pron. The task is to generate synopses for movie titles that do not exist. The synopses must be creative and appealing to both critics and the general audience, and must have literary value in and of themselves.

Here are some details of the novelist you will be competing with:
Patricio Pron (Rosario, December 9, 1975) is a writer and literary critic. Granta magazine selected him in 2010 as one of the 22 best young writers in Spanish. He won the twenty-second Alfaguara Novel Prize in 2019 for his work Mañana tendremos otros nombres."""
user_prompt = """The proposed title is: "{titulo}". Please write a synopsis of about 600 words for that title that meets the above specifications."""

In [6]:
df = pd.read_csv('data/sinopsis.csv')

In [7]:
all_texts = []
all_titles = df['title'].tolist()
all_titles_english = df['english_title'].tolist()
len_all_titles = len(all_titles)

for i, title in enumerate(all_titles_english):
    start_time = time.time()
    print(title)
    text = generate_gpt4(title, system_prompt, user_prompt)
    all_texts.append({
        'title':all_titles[i],
        'english_title':title,
        'text':text
    })
    print(text)
    save_json(path='gpt_sinopsis_english.json', data=all_texts)
    print_reaming_time(start_time=start_time, current_iterations=i, total_trainings=len_all_titles)

Every picture from when we were young
Title: "Every Picture from When We Were Young"

Synopsis:

In the dimly lit attic of an ancestral home in a town suspended in time, Julian discovers a chest brimming with aged photographs. These aren't just any photographs; they are windows into lives he never knew belonged to his lineage. As dust swirls in the shafts of sunlight piercing the room, each picture whispers secrets from the past, unveiling a tapestry of love, loss, rebellion, and reconciliation. "Every Picture from When We Were Young" is a cinematic journey through the epochs, tying together disparate generations through the immutable visibility of photographs.

Julian, a disillusioned photographer grappling with the ephemeral nature of modern relationships and art, is compelled by an inexplicable urge to return to his family's deserted estate after years of estrangement. The house, an edifice of memories and secrets, begins to unravel its stories through the photographs Julian uncover

In [ ]:
all_texts['text']

In [9]:
df_en = pd.DataFrame.from_records(all_texts)

In [12]:
pd.merge(df, df_en, on='title', how='outer').to_csv('generated_sinopsis.csv')